In [1]:
from time import time
import re
import string
import random

algorytm naiwny

In [2]:
def naive_string_matching(text, pattern, printt = True):
    t = time()
    for s in range(0, len(text) - len(pattern) + 1):
        if pattern == text[s:s + len(pattern)] and printt:
            print(f"Przesunięcie {s} jest poprawne")
    return time() - t

algorytm automatu skończonego

In [134]:
def fa_string_matching(text, pattern, printt = True):
    tt = time()
    q = 0
    delta = transition_table(pattern)

    t = time()
    
    for s in range(len(text)):
        q = delta[q].get(text[s], 0) # jesli nie ma znaku w tablicy przejscia to domyslnie wrzuca nam 0

        if (q == len(delta) - 1) and printt:
            print(f"Przesunięcie {s + 1 - q} jest poprawne")
            # s + 1 - ponieważ przeczytaliśmy znak o indeksie s, więc przesunięcie jest po tym znaku
    return time() - tt, time() - t 


def check_pattern(pattern, k, q, a):
    t = pattern[:q] + a
#     print("pattern + a",t)
#     if k < q:
#         return False
    
    for i in range (q - k + 2):
#         print(k, q, a, i, t[i:i+k], pattern[:k])
        if t[i:i+k] == pattern[:k]:
            return True
    
    return False

def transition_table(pattern):
    result = []

    alphabet = set(pattern)

    for q in range(len(pattern) + 1):
        result.append({})
        for a in alphabet:
            k = min(len(pattern), q + 1)
            while True:   
                
                print(pattern[:k], pattern[:q]+a)
                print(re.search(f"{pattern[:k]}$", pattern[:q] + a)) # dolar - do konca tego co jest pozniej
                print(check_pattern(pattern, k, q, a))
                print()
                
                if re.search(f"{pattern[:k]}$", pattern[:q] + a):
                    break
                
#                 if check_pattern(pattern, k, q, a):
#                     break
                    
                k = k - 1

            result[q][a] = k
    return result

In [136]:
pattern = "ACACAGA"
transition_table(pattern)

A A
<re.Match object; span=(0, 1), match='A'>
True

A C
None
False

 C
<re.Match object; span=(1, 1), match=''>
True

A G
None
False

 G
<re.Match object; span=(1, 1), match=''>
True

AC AA
None
False

A AA
<re.Match object; span=(1, 2), match='A'>
True

AC AC
<re.Match object; span=(0, 2), match='AC'>
True

AC AG
None
False

A AG
None
True

 AG
<re.Match object; span=(2, 2), match=''>
True

ACA ACA
<re.Match object; span=(0, 3), match='ACA'>
True

ACA ACC
None
False

AC ACC
None
True

A ACC
None
True

 ACC
<re.Match object; span=(3, 3), match=''>
True

ACA ACG
None
False

AC ACG
None
True

A ACG
None
True

 ACG
<re.Match object; span=(3, 3), match=''>
True

ACAC ACAA
None
False

ACA ACAA
None
True

AC ACAA
None
True

A ACAA
<re.Match object; span=(3, 4), match='A'>
True

ACAC ACAC
<re.Match object; span=(0, 4), match='ACAC'>
True

ACAC ACAG
None
False

ACA ACAG
None
True

AC ACAG
None
True

A ACAG
None
True

 ACAG
<re.Match object; span=(4, 4), match=''>
True

ACACA ACACA
<re.Match ob

[{'A': 1, 'C': 0, 'G': 0},
 {'A': 1, 'C': 2, 'G': 0},
 {'A': 3, 'C': 0, 'G': 0},
 {'A': 1, 'C': 4, 'G': 0},
 {'A': 5, 'C': 0, 'G': 0},
 {'A': 1, 'C': 4, 'G': 6},
 {'A': 7, 'C': 0, 'G': 0},
 {'A': 1, 'C': 2, 'G': 0}]

algorytm kmp (Knutha-Morrisa-Pratta)

In [67]:
def kmp_string_matching(text, pattern, printt = True):
    tt = time()
    pi = prefix_function(pattern)
    t = time()
    q = 0
    for i in range(len(text)):
        while q != 0 and pattern[q] != text[i]:
            q = pi[q - 1]

        if pattern[q] == text[i]:
            q = q + 1
        if q == len(pattern):
            if printt:
                print(f"Przesunięcie {i + 1 - q} jest poprawne")
            q = pi[q - 1]
    return time() - tt, time() - t

def prefix_function(pattern):
    pi = [0] * len(pattern)
    k = 0
    for q in range(1, len(pattern)):
        while k != 0 and pattern[k] != pattern[q]:
            k = pi[k - 1]

        if pattern[k] == pattern[q]:
            k = k + 1
        pi[q] = k
    return pi

Porównanie algorytmów

In [68]:
def speed_tests(text, pattern):
    naive = naive_string_matching(text, pattern, False)
    print("naive:        ", naive)
    print("fa")
    fa_total, fa_matching = fa_string_matching(text, pattern, False)
    print("total:        ", fa_total)
    print("matching:     ", fa_matching)
    print("preprocessing:", fa_total - fa_matching)
    fa_pre = fa_total - fa_matching
    
    print("kmp")
    kmp_total, kmp_matching = kmp_string_matching(text, pattern, False)
    print("total:        ", kmp_total)
    print("matching:     ", kmp_matching)
    print("preprocessing:", kmp_total - kmp_matching)
    kmp_pre = kmp_total - kmp_matching
    
    print()
    print("naive / fa matching: ", naive / fa_matching)
    print("naive / kmp matching:", naive / kmp_matching)
    print()
    print("fa preprocessing / kmp preprocessing:", fa_pre/kmp_pre)

In [69]:
text = open("ustawa", encoding="utf8").read()
pattern = "art"

In [83]:
speed_tests(text, pattern)

naive:         1.1205673217773438e-05
fa
<re.Match object; span=(0, 1), match='a'>
True

None
True

<re.Match object; span=(1, 1), match=''>
True

<re.Match object; span=(0, 2), match='aa'>
True

None
True

None
True

<re.Match object; span=(2, 2), match=''>
True

<re.Match object; span=(0, 3), match='aaa'>
True

None
True

None
True

None
True

<re.Match object; span=(3, 3), match=''>
True

<re.Match object; span=(0, 4), match='aaaa'>
True

None
True

None
True

None
True

None
True

<re.Match object; span=(4, 4), match=''>
True

<re.Match object; span=(0, 5), match='aaaaa'>
True

None
True

None
True

None
True

None
True

None
True

<re.Match object; span=(5, 5), match=''>
True

<re.Match object; span=(0, 6), match='aaaaaa'>
True

None
True

None
True

None
True

None
True

None
True

None
True

<re.Match object; span=(6, 6), match=''>
True

<re.Match object; span=(0, 7), match='aaaaaaa'>
True

None
True

None
True

None
True

None
True

None
True

None
True

None
True

<re.Match ob

In [74]:
text = "a"*100
pattern = "aaaaaaabbbbbbbb"
# print(text, pattern)
speed_tests(text, pattern)

naive:         1.1205673217773438e-05
fa
<re.Match object; span=(0, 1), match='a'>
True
True
<re.Match object; span=(1, 1), match=''>
True
<re.Match object; span=(0, 2), match='aa'>
True
True
True
<re.Match object; span=(2, 2), match=''>
True
<re.Match object; span=(0, 3), match='aaa'>
True
True
True
True
<re.Match object; span=(3, 3), match=''>
True
<re.Match object; span=(0, 4), match='aaaa'>
True
True
True
True
True
<re.Match object; span=(4, 4), match=''>
True
<re.Match object; span=(0, 5), match='aaaaa'>
True
True
True
True
True
True
<re.Match object; span=(5, 5), match=''>
True
<re.Match object; span=(0, 6), match='aaaaaa'>
True
True
True
True
True
True
True
<re.Match object; span=(6, 6), match=''>
True
<re.Match object; span=(0, 7), match='aaaaaaa'>
True
True
True
True
True
True
True
True
<re.Match object; span=(7, 7), match=''>
True
True
<re.Match object; span=(1, 8), match='aaaaaaa'>
True
<re.Match object; span=(0, 8), match='aaaaaaab'>
True
True
True
True
True
True
True
True


In [ ]:
text = ''.join(random.choice("abc") for i in range(500000))
pattern = ''.join(random.choice("aaaaaaab") for i in range(10))
text = "a"*100
pattern = "aaaaaaaaaabcdefghijklmaaaaa"
# print(text, pattern)
speed_tests(text, pattern)